<a href="https://colab.research.google.com/github/SingamSaiVaraPrasad/my_projects/blob/main/DQN_frozenLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 11.1 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
import torch as torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import random
import nn.functional as F

In [ ]:
class deepQnet(nn.Module):
                                             # when ever we define a class we need to have __init__
  def __init_(self,nInput,nHidden,nOutput):
    super().__init__()
    self.fc1=nn.Linear(nInput,nHidden)
    self.outfcn=nn.Linear(nHidden,nOutput)
  def forwardFcn(self,x_inp):                #this calculates output for an input through the network
    self.x_inp=F.ReLU(self.fc1(x_inp))
    x_inp=self.outfcn(x_inp)
    return x_inp

In [ ]:
class replay_buffer_memory():
  def __init__(self,maxlen):
    self.memory=deque([],maxlen=maxlen)   # double ended queue stores the states rewards nextstates actions
  def addData(self,transition):
    self.memory.append(transition)
  def sample(self,sampSize):
    return random.sample(self.memory,sampSize)     # gives random batch of given size
  def __len__(self):
    return len(self.memory)


In [ ]:
class lake():
  learning_rate=0.001              # alpha for the equation of Temporal difference
  disc_factor=0.9                   #gamma
  replay_mem=1000                   # memory of replay buffer
  batch_sz=32                       # buffer size each time
  sync_rate=10                      # time steps after which sync of target and policy networks sync

  lossfcn=nn.MSEloss()
  optimizer=none
  ACTIONS=['L','R','U','D']
  def train(self,episodes,render=False,is_slippery=False):
    env=gym.make('FrozenLake-v1',map_name="4x4",is_slippery=is_slipper,render_mode='human' if render else none)
    num_states=env.observation_space.n
    num_actions=env.action_space.n

    epsilon=1 #100% random actions
    memory=replay_buffer_memory(self.replay_mem) #creating an object of memory class for lake object

    # create a target and policy network
    policy_net=deepQnet(num_states,num_states,num_actions)
    target_net=deepQnet(num_states,num_states,num_actions)
    target_net.load_state_dict(policy_net.state_dict()) ##make target and policy networks same

    self.optimizer=torch.optim.Adam(policy_net.parameters(),self.learning_rate)
    rewards=np.zeros(episodes) # rewards are stored in episodes initialized to 0

    #epsilon decay has to be tracked
    epsi=[]
    step_cnt=0;
    for i in range(episodes):
      env.reset()[0]
      terminated=False
      truncated=False
      while((not terminated) and (not truncated) ):
        if random.random()<epsilon:
          action=env.action_space.sample()
        else
          with torch.no_grad:
            action=policy_net(self.actions_to_dqn(state,num_states)).argmax().item()  #gives current state and total states to DQN
        nxt_state,reward,truncated,terminated,_= env(action)
        memory.addData(state,nxt_state,action,reward,terminated)
        state=nxt_state
        step_cnt+=1
        if reward==1:
          rewards[i]=1
        if(len(memory)>batch_sz and np.sum(rewards)>0): # only if the batch size is met and rewards are received
          mini_bactch=memory.sample(self.batch_sz)
          self.optim(mini_batch,policy_net,target_net) #a function

      def optim(self,mini_batch,policy_net,target_net):
        current_q_list=[]
        current_targetQlist=[]
        for state,nxt_state,action,reward,terminated in mini_batch:
          if terminated:
            target=reward
          else:
            with torch.no_grad:
                target=reward+disc_factor*target_net(self.actions_to_dqn(state,num_states)).max()
          current_q=policy_net(self.actions_to_dqn(state,num_states))
          current_q_list.append(current_q)
          target_q=target_net(self.actions_to_dqn(state,num_states))
          target_q[action]=target                        # for the current tupple, it has an action, for that action we are finding the bellman equation using above line
          current_targetQlist.append(current_target_q)   #then we put them into list after finding new target for current action
                                                         # this two lists are sent to update weights using back propogation
        loss=self.(torch.stack(current_q_list),torch.stack(current_targetQlist))
        self.optimizer.zero_grad()                       #initially gradient of all weights are made zero other wise they accumulate
        loss.backward()                                  #calculates gradient and stores
        self.optimizer.step()                            #updates the weights



      def actions_to_dqn(self,stt,num_stts):
        arr=tensor.zeros(num_stts)
        arr[stt]=1
        return arr





